## **Notebook PC1_Ativ5**
## Convolutional classifier for the MNIST database.
**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Arthur Felipe dos Santos Fernandes


In [ ]:
import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Reshape to be [samples][width][height][channels]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                                 activation='relu',
                                 input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.summary()
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)
print("test loss, test acc:", results)
model_json = model.to_json()
json_file = open("model_CNN.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_CNN.h5")
print("Model saved to disk")
os.getcwd()

In [4]:
import keras
def get_compiled_model():
    # Make a simple 2-layer densely-connected neural network.
    inputs = keras.Input(shape=(784,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model


def get_dataset():
    batch_size = 32
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )


# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# Test the model on all available devices.
model.evaluate(test_dataset)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1
Epoch 1/20
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2282 - sparse_categorical_accuracy: 0.9316 - val_loss: 0.1260 - val_sparse_categorical_accuracy: 0.9617
Epoch 2/20
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0968 - sparse_categorical_accuracy: 0.9710 - val_loss: 0.0917 - val_sparse_categorical_accuracy: 0.9725
Epoch 3/20
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0615 - sparse_categorical_accuracy: 0.9811 - val_loss: 0.0984 - val_sparse_categorical_accuracy: 0.9721
Epoch 4/20
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0443 - sparse_categorical_accuracy: 0.9863 - val_loss: 0.0947 - val_sparse_categorical_accuracy: 0.9747
Epoch 5/20
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0359 - sparse_categorical_accuracy: 0.9882 - val_loss: 0.0975 - val_spa

[0.1381690800189972, 0.9775000214576721]

In [2]:
import pandas as pd

# Crie uma lista com os nomes dos modelos
modelos = [ 'LC', 'ELM', 'MLP', 'CNN']

# Crie uma lista com as acurácias dos modelos
acuracias = [0.8642,
             0.9413,
             0.9778,
             0.9775
]

# Crie um dataframe do pandas com as informações
df = pd.DataFrame({'Modelos': modelos, 'Acurácia': acuracias})

# Imprima a tabela
print(df)


  Modelos  Acurácia
0      LC    0.8642
1     ELM    0.9413
2     MLP    0.9778
3     CNN    0.9775
